In [ ]:
import pandas as pd
import datetime
from datetime import timedelta
import pickle
import numpy as np
import matplotlib.pyplot as plt

1. **经度lng，对应h；维度lat，对应w**: grid[lat.ind][lng.ind]
2. 由于w和h相等，代码中没有对他们进行区分。

# Build Grid

## Find boundary of grid

In [ ]:
find_boundary_of_gride_mode=False

if find_boundary_of_gride_mode==True:
    # tol_len is 3486*40000
    reader = pd.read_csv('H:\时空预测\code1_TaxiDatasetPropose\Chicago Taxi\Taxi_Trips\Taxi_Trips.csv', chunksize=40000)

    max_lat,min_lat,max_lng,min_lng=-1000, 1000, -1000, 1000
    for chunk in reader:
        max_lat = max(max_lat, max(chunk['Pickup Centroid Latitude']))
        min_lat = min(min_lat, min(chunk['Pickup Centroid Latitude']))
        max_lng = max(max_lng, max(chunk['Pickup Centroid Longitude']))
        min_lng = min(min_lng, min(chunk['Pickup Centroid Longitude']))
    print(max_lat, min_lat, max_lng, min_lng)
else:
    max_lat,min_lat,max_lng,min_lng=42.021223593, 41.650221676, -87.529950466, -87.913624596

## define grid (w $\times$ h)

In [ ]:
w,h=30,30

def get_grid_index(lat, lng):
    print(lat, lng)
    print(min_lat, max_lat, min_lng, max_lng)
    assert lat >= min_lat and lat <= max_lat
    assert lng >= min_lng and lng <= max_lng
    unit_lat=(max_lat-min_lat)/h
    unit_lng=(max_lng-min_lng)/w
    # lower_left of grid[i][j], 0<=i,j<30
    return int((lat-min_lat)/unit_lat) if lat<max_lat else h-1, int((lng-min_lng)/unit_lng) if lng<max_lng else w-1

In [ ]:
unit_lat=(max_lat-min_lat)/h
unit_lng=(max_lng-min_lng)/w
print(unit_lat, unit_lng)

## build TrafficGrid

In [ ]:
begin_time=datetime.datetime.strptime('2013-01-01 12:00:00 PM', '%Y-%m-%d %I:%M:%S %p')
end_time=datetime.datetime.strptime('2022-04-01 12:00:00 PM', '%Y-%m-%d %I:%M:%S %p')
td_15min = timedelta(days=0, seconds=0, microseconds=0,milliseconds=0, minutes=0, hours=1, weeks=0)
time_len = int((end_time-begin_time)/td_15min +1)
print(time_len)
# 此处为运行出来的结果
TimeRange=['2013-01-01 12:00:00', '2022-04-01 12:00:00']

In [ ]:
def get_time_slot_index(dt):
    return int((dt-begin_time)/td_15min)
run=True
if run:
    # 2013-01-01 00:00:00	2022-02-28 22:45:00
    time_format ='%m/%d/%Y %I:%M:%S %p'
    TrafficGrid = np.zeros([time_len, w, h])
    reader = pd.read_csv('H:\时空预测\code1_TaxiDatasetPropose\Chicago Taxi\Taxi_Trips\Taxi_Trips.csv', chunksize=4000)
    for chunk in reader:
        chunk.drop(chunk[np.isnan(chunk['Pickup Centroid Latitude'])].index, inplace=True)
        for ind, row in chunk.iterrows():
            lat = row['Pickup Centroid Latitude']
            lng = row['Pickup Centroid Longitude']
            i, j = get_grid_index(lat, lng)
            try:
                dt = datetime.datetime.strptime(row['Trip Start Timestamp'], time_format)
                TrafficGrid[get_time_slot_index(dt), i, j] += 1
                # 共打印12305次
                if ind % 10000==0:
                    print(dt)
                    # print(TrafficGrid[get_time_slot_index(dt)])
            except:
                print('出错！')

In [ ]:
if run:
    las_has_data=TrafficGrid.shape[0]
    for i in range(TrafficGrid.shape[0]-1, 0, -1):
        if not(np.any(TrafficGrid[i])): # if all zeros, mean that there is no traffic data
            continue
        else:
            las_has_data = i+1
            break
    np.save("tol_arr.npy", TrafficGrid)
    np.save("arr_with_data.npy", TrafficGrid[0:las_has_data,:,:])
    print(TrafficGrid.shape[0], las_has_data)

In [ ]:
# 总共加了122,963,134，总共数据条数139,600,000
tg_sum = TrafficGrid.sum()
tg_sum 

In [ ]:
arr = np.load('arr_with_data.npy')
arr = np.sum(arr, axis=0)

## 绘制

In [ ]:
sm = False
save_fig = True
sfx = 'times'

### 绘制三维投影面

In [ ]:
def softmax(x):
    max = np.max(x,axis=1,keepdims=True) #returns max of each row and keeps same dims
    e_x = np.exp(x - max) #subtracts each row with its max value
    sum = np.sum(e_x,axis=1,keepdims=True) #returns sum of each row and keeps same dims
    f_x = e_x / sum 
    return f_x

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['font.sans-serif']=['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False # 用来正常显示负号


X_BOUND = [0, len(arr)]
Y_BOUND = [0, len(arr[0])]

#绘制3D图形
#def plot_3d(ax):\
ax = plt.axes(projection='3d')
X = np.array(list(range(len(arr))))
Y = np.array(list(range(len(arr[0]))))
X, Y = np.meshgrid(X, Y)#坐标矩阵，每个交叉点对应网格
Z = softmax(arr) if sm else arr
    # 绘制3D图形，X,Y,Z:2D数组形式的数据值
    #rstride:数组行距（步长大小）
    #cstride:数组列距(步长大小）
    #cmap:曲面块颜色映射
    #color:曲面块颜色
font2 = {'family': 'Times New Roman',
             'weight': 'normal',
             'size': 14,
             }
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm)
# ax.set_zlim(-5, 20)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z',font2)
ax.set_title('Ackley',font2)

#plt.yticks(fontproperties='Times New Roman', size=14)
#plt.xticks(fontproperties='Times New Roman', size=14)
    # 设置横纵坐标的名称以及对应字体格式
    #
plt.xlabel('x1', font2)
plt.ylabel('x2', font2)

ax.contourf(X, Y, Z, zdir='z',offset=-5)
if save_fig:
    plt.savefig(f'./{sfx}{"softmax_" if sm else ""}Ackley.png',dpi=1200)
plt.show()

if __name__ == "__main__":
    #绘制函数三维图像
    fig = plt.figure()
    ax = Axes3D(fig)
    #plot_3d(ax)


### 绘制3D柱状图

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
 
# 绘图设置
fig = plt.figure()
ax = fig.gca(projection='3d')  # 三维坐标轴
# X和Y的个数要相同
X = list(range(len(arr)))
Y = list(range(len(arr)))
Z = np.random.randint(0, 1000, len(X)*len(Y)) # 生成16个随机整数
# meshgrid把X和Y变成平方长度，比如原来都是4，经过meshgrid和ravel之后，长度都变成了16，因为网格点是16个
xx, yy = np.meshgrid(X, Y)  # 网格化坐标
X, Y = xx.ravel(), yy.ravel()  # 矩阵扁平化
# 设置柱子属性
height = np.zeros_like(Z) # 新建全0数组，shape和Z相同，据说是图中底部的位置
width = depth = 0.2 # 柱子的长和宽
# 颜色数组，长度和Z一致
c = ['r']*len(Z)
# 开始画图，注意本来的顺序是X, Y, Z, width, depth, height，但是那样会导致不能形成柱子，只有柱子顶端薄片，所以Z和height要互换
ax.bar3d(X, Y, height, width, depth, (softmax(arr) if sm else arr).reshape([w*h,]),  color=c, shade=False)  # width, depth, height
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
if save_fig:
    plt.savefig(f'./{sfx}{"softmax_" if sm else ""}3D_col.png', dpi=1200)

In [ ]:
for x in arr:
    for t in x:
        print(t, end='\t')
    print()

In [ ]:
arr_times = np.load('arr_with_data.npy')
print(arr_times.shape)
for i in range(len(arr_times)):
    if i % 10000==0:
        print(i)
    for j in range(30):
        for k in range(30):
            if arr_times[i][j][k] >= 1:
                arr_times[i][j][k] = 1
np.save("arr_hasdata_time.npy", arr_times)

In [ ]:
arr_sum = (np.sum(arr, axis=0))

In [ ]:

for x in arr_sum:
    for t in x:
        print(t, end='\t')
    print()

## Build ['Grid']['GridLatLng']

In [ ]:
max_lat,min_lat,max_lng,min_lng
unit_lat, unit_lng

GridLatLng = [[min_lat, min_lng]]
for i in range(1, w+1):
    for j in range(1, h+1):
        GridLatLng.append([min_lat+i*unit_lat, min_lng+i*unit_lng])


# Build TimeFitness、Time Range and LenTimeSlots

In [ ]:
TimeFitness = 15
TimeRange = [begin_time.strftime('%Y-%m-%d'), end_time.strftime('%Y-%m-%d')]
print(TimeRange)
LenTimeSlots = time_len

# Build Node

## find node

In [ ]:
data =np.load('arr_hasdata_time.npy')

threshold = $\alpha \times len(data)$

意味如果超过$(1-\alpha)$的时间没有数据，则不考虑

In [ ]:
print(len(data))
shld = 0.01 * len(data)
print(shld)
# shld = 10

l = 500

data_times = np.sum(data, axis=0)
lis = []
for i in range(len(data_times)):
    for j in range(len(data_times[0])):
        if data_times[i][j] > shld:
            lis.append([i,j,data_times[i][j]])
print(f'剩余网格个数：{len(lis)}')
lis = sorted(lis, key=lambda t: t[2], reverse=True)
ma, mi = int(lis[0][2]), int(lis[-1][2]) # 1, 57663
print(mi, ma)

# 统计剩下的所有数据中，长度为l的区间内的数据个数
cnts = []
for d in range(mi, ma, l):
    cnt = 0
    for t in lis:
        if t[2] >= d and t[2] < d+l:
            cnt+=1
    cnts.append(cnt)
plt.plot(range(mi, ma, l), cnts)

由于$\alpha$取0.01时剩余150个网格，每个网格划分$4\times4$的小网格，共有2400个节点

将需要$ 81049\times2400\times4 B=742MB$内存存储

## Build StationInfo

In [ ]:
div = 4

tiny_w = w * div
tiny_h = h * div
tiny_unit_lat=(max_lat-min_lat)/tiny_h
tiny_unit_lng=(max_lng-min_lng)/tiny_w


# 新定义 480*480 的小网格
def get_tiny_grid_index(lat, lng):
    assert lat >= min_lat and lat <= max_lat
    assert lng >= min_lng and lng <= max_lng
    return int((lat-min_lat)/tiny_unit_lat) if lat<max_lat else tiny_h-1, int((lng-min_lng)/tiny_unit_lat) if lng<max_lng else tiny_w-1

tiny_grid_to_nodeid = [[-1 for _ in range(tiny_w)] for _ in range(tiny_h)]

StationInfo = []

ind = 0
for t in lis:
    i, j = t[0], t[1]
    tiny_i = i*div
    tiny_j = j*div
    for k in range(div):
        for l in range(div):
            new_i = tiny_i+k
            new_j = tiny_j+l
            tiny_grid_to_nodeid[new_i][new_j] = ind
            StationInfo.append([hash((new_i, new_j)), TimeRange[0], 
            min_lat+tiny_i*tiny_unit_lat, min_lng+tiny_j*unit_lng, 
            f'node-({new_i},{new_j})'])
            ind+=1
print(ind)
print(StationInfo[:2])

In [ ]:
# del TrafficGrid
TrafficNode = np.zeros([time_len, ind])
print(time_len, ind)
# 2013-01-01 00:00:00	2022-02-28 22:45:00
time_format ='%m/%d/%Y %I:%M:%S %p'
reader = pd.read_csv('H:\时空预测\Chicago Taxi\Taxi_Trips\Taxi_Trips.csv', chunksize=4000)
for chunk in reader:
    chunk.drop(chunk[np.isnan(chunk['Pickup Centroid Latitude'])].index, inplace=True)
    for ind, row in chunk.iterrows():
        lat = row['Pickup Centroid Latitude']
        lng = row['Pickup Centroid Longitude']
        # print(lat, lng)
        i, j = get_tiny_grid_index(lat, lng)
        try:
            # print(i, j)
            dt = datetime.datetime.strptime(row['Trip Start Timestamp'], time_format)
            # print(dt, get_time_slot_index(dt))
            # print(tiny_grid_to_nodeid[i][j])
            TrafficNode[get_time_slot_index(dt)][tiny_grid_to_nodeid[i][j]] += 1
            # 共打印12305次
            if ind % 10000==0:
                print(dt)
                # print(TrafficGrid[get_time_slot_index(dt)])
        except:
            print('出错！')

In [ ]:
np.save("TrafficNode.npy", TrafficNode)

# Build Dict

In [ ]:
ds = {
    'TimeFitness':TimeFitness,
    'TimeRange':TimeRange,
    'Node':{
        'TrafficNode':None,
        'TrafficMonthlyInteraction':None,
        'StationInfo':None,
        'POI':[]
    },
    'Grid':{
        'TrafficGrid':TrafficGrid,
        'GridLatLng':GridLatLng,
        'POI':[]
    },
    'ExternalFeature':{
        
    }
}